In [6]:
import numpy as np
import pandas as pd
import gzip
import json
from tqdm import tqdm
import regex as re
import time
import gc
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
from itertools import combinations
import sys
sys.maxsize

9223372036854775807

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Extracting data from json.gz

In [ ]:
g = gzip.open('/content/drive/MyDrive/data_for_lab/RS_google_data/reviews.clean.json.gz', 'rb')

i,e  = 0, 0
pattern = r"'([0-9_\./\\-]*)'"
main_list = []

for l in g:
  try:
    l = l.decode('UTF-8')
    
    q1 = re.search('rating', l)
    r = int(float(re.findall("\d+\.\d+", l[q1.span()[1]:])[0]))
    q2 = re.search('gPlusPlaceId', l)
    u = int(re.findall("u'[0-9]*'", l[q2.span()[1]:])[0][2:-1])
    q3 = re.search('gPlusUserId', l)
    p = int(re.findall("u'[0-9]*'", l[q3.span()[1]:])[0][2:-1])

    main_list.append([p,u,r])
    if i % 100000 == 0:
      print(i,e)
      i += 1
    else:
      i += 1
  except:
    e += 1

0 0
100000 0
200000 0
300000 0
400000 0
500000 0
600000 0
700000 0
800000 0
900000 0
1000000 0
1100000 0
1200000 0
1300000 0
1400000 0
1500000 0
1600000 0
1700000 0
1800000 0
1900000 0
2000000 0
2100000 0
2200000 0
2300000 0
2400000 0
2500000 0
2600000 0
2700000 0
2800000 0
2900000 0
3000000 0
3100000 0
3200000 0
3300000 0
3400000 0
3500000 0
3600000 0
3700000 0
3800000 0
3900000 0
4000000 0
4100000 0
4200000 0
4300000 0
4400000 0
4500000 0
4600000 0
4700000 0
4800000 0
4900000 0
5000000 0
5100000 0
5200000 0
5300000 0
5400000 0
5500000 0
5600000 0
5700000 0
5800000 0
5900000 0
6000000 0
6100000 0
6200000 0
6300000 0
6400000 0
6500000 0
6600000 0
6700000 0
6800000 0
6900000 0
7000000 0
7100000 0
7200000 0
7300000 0
7400000 0
7500000 0
7600000 0
7700000 0
7800000 0
7900000 0
8000000 0
8100000 0
8200000 0
8300000 0
8400000 0
8500000 0
8600000 0
8700000 0
8800000 0
8900000 0
9000000 0
9100000 0
9200000 0
9300000 0
9400000 0
9500000 0
9600000 0
9700000 0
9800000 0
9900000 0
10000000 0
1010

In [ ]:
df = pd.DataFrame(np.array(main_list))
df.columns = ['gPlusPlaceId', 'gPlusUserId', 'rating']

In [ ]:
df.head()

,gPlusPlaceId,gPlusUserId,rating
0,100000010817154263736,108103314380004200232,3
1,100000010817154263736,108103314380004200232,3
2,100000010817154263736,108103314380004200232,3
3,100000010817154263736,108103314380004200232,3
4,100000010817154263736,108103314380004200232,3


In [ ]:
df.to_csv('/content/drive/MyDrive/data_for_lab/RS_google_data/user_item_rating.csv')

### There were some errors in userids. So getting all the proper userids.

In [ ]:
g = gzip.open('/content/drive/MyDrive/data_for_lab/RS_google_data/users.clean.json.gz', 'rb')
uset = set()
for l in g:
  l = l.decode('UTF-8')
  uset.add(int(l[-24:-3]))

In [ ]:
len(uset)

3747937

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_for_lab/RS_google_data/user_item_rating.csv')
test = list(set(df.gPlusUserId))

count = 0
rem = []
for i in test:
  if int(i) not in uset:
    count += 1
    rem.append(i)

In [ ]:
new_df = df[~df.gPlusUserId.isin(rem)]

In [ ]:
new_df.head()

,Unnamed: 0,gPlusPlaceId,gPlusUserId,rating
0,0,108103314380004200232,100000010817154263736,3
1,1,102194128241608748649,100000013500285534661,5
2,2,101409858828175402384,100000021336848867366,5
3,3,101477177500158511502,100000021336848867366,5
4,4,106994170641063333085,100000021336848867366,4


In [ ]:
len(np.unique(new_df.gPlusUserId))

3740866

In [ ]:
len(np.unique(new_df.gPlusPlaceId))

2818575

In [ ]:
new_df.to_csv('/content/drive/MyDrive/data_for_lab/RS_google_data/user_item_rating_final.csv')

# Creating user-item rating martrix.
### Taking users with more than 15 reviews and places with more than 15 reviews.

In [ ]:
new_df = pd.read_csv('/content/drive/MyDrive/data_for_lab/RS_google_data/user_item_rating_final.csv')
new_df = new_df.drop(['Unnamed: 0.1'], axis=1)

In [ ]:
df_gb = new_df.groupby('gPlusUserId')

total = len(np.unique(new_df.gPlusUserId))

min_reviews_present = 15
new_df_red = pd.DataFrame()
temp = pd.DataFrame()
print('Starting......')
pbar = tqdm(total=100, position=0, leave=True)
c = 0
for i in df_gb:
  
  if c == int(total/100):
    pbar.update(1)
    c = 0
  else:
    c += 1

  if len(i[1]) >= min_reviews_present:
    temp = temp.append(i[1], ignore_index=True)

    if len(temp) > 2000:
      new_df_red = new_df_red.append(temp, ignore_index=True)
      temp = pd.DataFrame()

pbar.update(1)
pbar.close()

print(new_df_red.shape)


Starting......


100%|██████████| 100/100 [03:28<00:00,  2.09s/it]

(2073556, 4)


In [ ]:
df_gb_2 = new_df_red.groupby('gPlusPlaceId')
total = len(np.unique(new_df_red.gPlusPlaceId))

min_reviews_present = 15
new_df_red_2 = pd.DataFrame()
temp = pd.DataFrame()
print('Starting......')
pbar = tqdm(total=100, position=0, leave=True)
c = 0
for i in df_gb_2:
  
  if c == int(total/100):
    pbar.update(1)
    c = 0
  else:
    c += 1

  if len(i[1]) >= min_reviews_present:
    temp = temp.append(i[1], ignore_index=True)

    if len(temp) > 2000:
      new_df_red_2 = new_df_red_2.append(temp, ignore_index=True)
      temp = pd.DataFrame()

pbar.update(1)
pbar.close()

print(new_df_red_2.shape)

Starting......


100%|██████████| 100/100 [00:47<00:00,  2.09it/s]

(281420, 4)


In [ ]:
len(np.unique(new_df_red_2.gPlusPlaceId)), len(np.unique(new_df_red_2.gPlusUserId))

(9766, 31623)

### Creating pivot table.

In [ ]:
rows, row_pos = np.unique(new_df_red_2.iloc[:, 2], return_inverse=True)
cols, col_pos = np.unique(new_df_red_2.iloc[:, 1], return_inverse=True)

pivot_table = np.zeros((len(rows), len(cols)), dtype='object')
pivot_table[row_pos, col_pos] = new_df_red_2.iloc[:, 3]

pivot_table.shape

(31623, 9766)

In [ ]:
pivot_df = pd.DataFrame(pivot_table, columns = cols, index = rows)
pivot_df

,100000418786238226598,100003306461961201673,100006042449520085772,100006647043412576027,100006882617735881358,100008293862971697418,100009296329313456387,100014438093740548910,100018168642444114921,100018779661168745263,...,118406219806619147119,118406957693362928195,118408091801984983805,118408308120993277213,118412280948887004355,118412529402110044890,118415291419685713017,118416014053319084823,118418639274455787853,118420416172234080253
100000242929724813348,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100001176398972193069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100001564871364169453,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100001643728576657268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100002289762977414196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114670844158895297265,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114671116312426288270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114671780747519572291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114672393604744185833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pivot_df.to_csv('/content/drive/MyDrive/data_for_lab/RS_google_data/rating_matrix_user15_place15.csv')

# Creating Models

## 1) Neighbourhood model with baseline estimates. (without Implicit feedback)

In [ ]:
class NM_wo_implicit():
  def __init__(self, arr, k_neighbours, lmda1, lmda4, gamma, iter):
    self.arr = arr
    self.sim_mat = self.sim_mat_cal()

    self.bu = np.random.rand(arr.shape[0])
    self.bi = np.random.rand(arr.shape[1])
    self.u = np.mean(arr)
    self.w = np.random.rand(arr.shape[1],arr.shape[1])
    self.c = np.random.rand(arr.shape[1],arr.shape[1])

    self.b_uj = np.random.rand(arr.shape[0],arr.shape[1])
    self.b_u = np.random.rand(arr.shape[0])
    self.b_i = np.random.rand(arr.shape[1])

    self.k_neighbours = k_neighbours
    self.lmda1 = lmda1
    self.lmda4 = lmda4
    self.gamma = gamma
    self.iter = iter

    self.known_ratings = np.argwhere(self.arr != 0)

    self.baseline_estimates()

  def baseline_estimates(self):
    for _ in range(self.iter):
      for i in self.known_ratings:
        self.b_u[i[0]] = self.b_u[i[0]] + (self.arr[i[0],i[1]] - self.u - self.b_u[i[0]] - self.b_i[i[1]]) - self.lmda1 * self.b_u[i[0]]
        self.b_i[i[1]] = self.b_i[i[1]] + (self.arr[i[0],i[1]] - self.u - self.b_u[i[0]] - self.b_i[i[1]]) - self.lmda1 * self.b_i[i[1]]
    
    for i in range(self.b_uj.shape[0]):
      self.b_uj[i,:] = self.b_u[i] + self.b_i + self.u

  def sim_mat_cal(self):
    temp = np.corrcoef(self.arr)
    sim_mat = np.where(np.isnan(temp), 0, temp)

    for i in range(sim_mat.shape[0]):
      for j in range(sim_mat.shape[0]):
        n_ij = len(set(np.argwhere(self.arr[:,i] != 0)[:,0]) & set(np.argwhere(self.arr[:,j] != 0)[:,0]))
        sim_mat[i,j] *= (n_ij / (n_ij + 100))

    return sim_mat

  def fit(self):
    for epoch in range(self.iter):
      total,count,err = len(self.known_ratings), 1, []
    
      for i in self.known_ratings:
        RK_iu = list(set(np.argsort(self.sim_mat[i[1]][1:])[::-1][:self.k_neighbours]) & set(self.known_ratings[self.known_ratings[:,0] == i[0]][:,1]))
        s_temp = 0

        if RK_iu:
          for j in RK_iu:
            s_temp += (self.arr[i[0], j] - self.b_uj[i[0], j]) * self.w[i[1], j]

          s_temp = len(RK_iu) ** (-1/2) * s_temp

        r_hat = self.u + self.bu[i[0]] + self.bi[i[1]] + s_temp

        # update
        error = self.arr[i[0], i[1]] - r_hat
        err.append(error)

        self.bu[i[0]] += self.gamma * (error - self.lmda4 * self.bu[i[0]])
        self.bi[i[1]] += self.gamma * (error - self.lmda4 * self.bi[i[1]])

        if RK_iu:
          for j in RK_iu:
            self.w[i[1],j] += self.gamma * ((len(RK_iu)**(-1/2) * error * (self.arr[i[0], j] - self.b_uj[i[0], j])) - self.lmda4 * self.w[i[1], j])
        
        if count % int(total / 10) == 0:
          s = 'Epoch ' + str(epoch) + ' [' + '===' * int((count+1) / total * 10) + '>' + '   ' * (10 - int((count+1) / total * 10)) + ']' + '  Error (RMSE): ' + str(np.sqrt(np.mean(err)))
          print(f"\r{s}", end="")
        
        count += 1
      print()

  def predict_index(self, user_index, top_k):
    rec_index, ret_list = [], []

    temp1 = self.known_ratings[self.known_ratings[:,0] == user_index][:,1]
    for i in range(self.arr.shape[1]):
      RK_iu = list(set(np.argsort(self.sim_mat[i][1:])[::-1][:self.k_neighbours]) & set(self.known_ratings[self.known_ratings[:,0] == user_index][:,1]))

      s_temp = 0

      if RK_iu:
        for j in RK_iu:
          s_temp += (self.arr[user_index, j] - self.b_uj[user_index, j]) * self.w[i, j]

        s_temp = len(RK_iu) ** (-1/2) * s_temp

      r_hat = self.u + self.bu[user_index] + self.bi[i] + s_temp

      rec_index.append(r_hat)

    rev_sorted_index = np.argsort(rec_index)[::-1]

    for i in rev_sorted_index:
      if i not in temp1:
        ret_list.append(i)

      if len(ret_list) == top_k:
        break
    
    return ret_list
    

### Testing the function.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_for_lab/RS_google_data/rating_matrix_user15_place15.csv')
df = df.set_index('Unnamed: 0')
arr = np.array(df.iloc[:1000,:1000])

In [ ]:
m = NM_wo_implicit(arr, 100, 0.02, 0.002, 0.005, 10)

/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [ ]:
m.fit()

Epoch 0 [==============================>]  Error (RMSE): 1.755479500129229
Epoch 1 [==============================>]  Error (RMSE): 1.7238310157913606
Epoch 2 [==============================>]  Error (RMSE): 1.6932773327122561
Epoch 3 [==============================>]  Error (RMSE): 1.6637667510673122
Epoch 4 [==============================>]  Error (RMSE): 1.6352502843447398
Epoch 5 [==============================>]  Error (RMSE): 1.607681518019065
Epoch 6 [==============================>]  Error (RMSE): 1.5810164748766162
Epoch 7 [==============================>]  Error (RMSE): 1.555213486743704
Epoch 8 [==============================>]  Error (RMSE): 1.5302330723749953
Epoch 9 [==============================>]  Error (RMSE): 1.5060378212658463


In [ ]:
## Here we are predicting the index of place in pivot arr created from pivot table.
##  From this place id can be obtained and place name can be predicted. 

for i in [2,5,11,99, 235]:
  print(m.predict_index(i, 10)) # i is the index and 10 means top 10

[222, 165, 482, 581, 43, 395, 792, 274, 963, 424]
[222, 165, 482, 581, 43, 395, 792, 274, 963, 424]
[222, 165, 482, 581, 43, 395, 792, 274, 963, 424]
[222, 165, 482, 581, 43, 395, 792, 274, 963, 424]
[222, 165, 51, 792, 875, 43, 285, 584, 419, 395]


## 2) SVD with baseline estimates (without impicit)

In [ ]:
class MF_wo_implicit():
  def __init__(self, arr, k, lmda4, gamma, iter):
    self.arr = arr

    self.bu = np.random.rand(arr.shape[0])
    self.bi = np.random.rand(arr.shape[1])
    self.u = np.mean(arr)
    self.p = np.random.rand(arr.shape[0],k)
    self.q = np.random.rand(k,arr.shape[1])

    self.k = k
    self.lmda4 = lmda4
    self.gamma = gamma
    self.iter = iter

    self.known_ratings = np.argwhere(self.arr != 0)


  def fit(self):
    for epoch in range(self.iter):
      total,count,err = len(self.known_ratings), 1, []
    
      for i in self.known_ratings:
        r_hat = self.u + self.bu[i[0]] + self.bi[i[1]] + np.dot(self.p[i[0],:], self.q[:, i[1]])

        # update
        error = self.arr[i[0], i[1]] - r_hat
        err.append(error)

        self.bu[i[0]] += self.gamma * (error - self.lmda4 * self.bu[i[0]])
        self.bi[i[1]] += self.gamma * (error - self.lmda4 * self.bi[i[1]])

        for j in range(self.k):
          self.p[i[0],j] += self.gamma * (error * self.q[j,i[1]] + self.lmda4 * self.p[i[0], j])
          self.q[j, i[1]] += self.gamma * (error * self.p[i[0],j] + self.lmda4 * self.q[j, i[1]])
        
        if count % int(total / 10) == 0:
          s = 'Epoch ' + str(epoch) + ' [' + '===' * int((count+1) / total * 10) + '>' + '   ' * (10 - int((count+1) / total * 10)) + ']' + '  Error (RMSE): ' + str(np.sqrt(abs(np.mean(err))))
          print(f"\r{s}", end="")
        
        count += 1
      print()

  def predict_index(self, user_index, top_k):
    rec_index, ret_list = [], []

    temp1 = self.known_ratings[self.known_ratings[:,0] == user_index][:,1]
    for i in range(self.arr.shape[1]):
      r_hat = self.u + self.bu[user_index] + self.bi[i] + np.dot(self.p[user_index,:], self.q[:,i])

      rec_index.append(r_hat)

    rev_sorted_index = np.argsort(rec_index)[::-1]

    for i in rev_sorted_index:
      if i not in temp1:
        ret_list.append(i)

      if len(ret_list) == top_k:
        break
    
    return ret_list

In [ ]:
m = MF_wo_implicit(arr,400,0.002,0.005,10)
m.fit()

Epoch 0 [==============================>]  Error (RMSE): 7.3435703268223245
Epoch 1 [==============================>]  Error (RMSE): 2.326416003672824
Epoch 2 [==============================>]  Error (RMSE): 1.1220969831247738
Epoch 3 [==============================>]  Error (RMSE): 0.7431773694646442
Epoch 4 [==============================>]  Error (RMSE): 0.5237073719048334
Epoch 5 [==============================>]  Error (RMSE): 0.3782915673357733
Epoch 6 [==============================>]  Error (RMSE): 0.2759147604030514
Epoch 7 [==============================>]  Error (RMSE): 0.20113323127930846
Epoch 8 [==============================>]  Error (RMSE): 0.14484672008721175
Epoch 9 [==============================>]  Error (RMSE): 0.10117439303440409


In [ ]:
m.predict_index(1, 10)

[551, 62, 808, 951, 641, 500, 711, 101, 608, 258]

In [ ]:
m.predict_index(10, 10)

[526, 653, 458, 302, 618, 679, 344, 434, 345, 872]

In [ ]:
for i in [2,5,11,99, 235]:
  print(m.predict_index(i, 10))

[641, 540, 113, 315, 721, 842, 872, 22, 608, 618]
[872, 252, 641, 592, 618, 608, 653, 422, 842, 283]
[249, 47, 737, 641, 618, 950, 309, 84, 343, 569]
[678, 842, 641, 315, 540, 493, 618, 598, 754, 747]
[872, 315, 641, 678, 919, 925, 99, 47, 618, 429]


### Comparision of neighbourhood vs svd approach


In [ ]:
#  neighbourhood approach                                   # svd approach
# [222, 165, 482, 581, 43, 395, 792, 274, 963, 424]         # [641, 540, 113, 315, 721, 842, 872, 22, 608, 618]
# [222, 165, 482, 581, 43, 395, 792, 274, 963, 424]         # [872, 252, 641, 592, 618, 608, 653, 422, 842, 283]
# [222, 165, 482, 581, 43, 395, 792, 274, 963, 424]         # [249, 47, 737, 641, 618, 950, 309, 84, 343, 569]
# [222, 165, 482, 581, 43, 395, 792, 274, 963, 424]         # [678, 842, 641, 315, 540, 493, 618, 598, 754, 747]
# [222, 165, 51, 792, 875, 43, 285, 584, 419, 395]          # [872, 315, 641, 678, 919, 925, 99, 47, 618, 429]

# Note: above are top 10 predicted indeces of places.
# From above we can see that neighbourhood approach gives almost same result for many users. Only few users with
# high number of ratings get different ratings. Whereas svd approach gives different result for every user based 
# on his/her ratings. Also impicit feedback were not used in any of the above.

## Trying SVD approach with higher dimension data.

In [ ]:
arr = np.array(df.iloc[:5000,:5000])
m = MF_wo_implicit(arr,400,0.002,0.005,10)
m.fit()

Epoch 0 [===========================>   ]  Error (RMSE): 4.968773724240454
Epoch 1 [===========================>   ]  Error (RMSE): 2.222248337769308
Epoch 2 [===========================>   ]  Error (RMSE): 1.1314795527805537
Epoch 3 [===========================>   ]  Error (RMSE): 0.7512093822522573
Epoch 4 [===========================>   ]  Error (RMSE): 0.5307122384782539
Epoch 5 [===========================>   ]  Error (RMSE): 0.3872232230614622
Epoch 6 [===========================>   ]  Error (RMSE): 0.2887533506316657
Epoch 7 [===========================>   ]  Error (RMSE): 0.21900652550026273
Epoch 8 [===========================>   ]  Error (RMSE): 0.16842994202874959
Epoch 9 [===========================>   ]  Error (RMSE): 0.13099364862815732


In [ ]:
for i in [2,5,11,99, 235]:
  print(m.predict_index(i, 10))

[3184, 2766, 3871, 3505, 2732, 3987, 1155, 2862, 430, 3478]
[3246, 3505, 1357, 1001, 4095, 3363, 2862, 3506, 430, 3394]
[986, 2127, 868, 3146, 2353, 3381, 3831, 2727, 4101, 4390]
[1897, 3246, 3184, 3061, 1136, 3987, 2117, 3506, 739, 2766]
[3796, 2010, 4680, 3952, 4316, 2639, 3792, 2862, 3794, 3061]


In [ ]:
# for user-index 2
# with 1000 user-item: [641, 540, 113, 315, 721, 842, 872, 22, 608, 618]
# with 5000 user-item: [3184, 2766, 3871, 3505, 2732, 3987, 1155, 2862, 430, 3478]

# checking the ratings of the user 2
c = 0
for i in list(arr[2,:]):
  if i != 0:
    print(c, i)
  c += 1

1972 3


In [ ]:
# user 2 rated only one item at position 1972.
# let's find the similarity of item 1972 with first item of 1000 user-item prediction
# and 5000 user-item prediction.

pearsonr(arr[:,1972], arr[:,641])[0], pearsonr(arr[:,1972], arr[:,3184])[0]

(-0.0009474917741406492, -0.0005106086083061693)

### Changing the value of k

In [ ]:
# k = 500
arr = np.array(df.iloc[:5000,:5000])
m2 = MF_wo_implicit(arr,500,0.002,0.001,10)
m2.fit()

Epoch 0 [===========================>   ]  Error (RMSE): 7.757643700354244
Epoch 1 [===========================>   ]  Error (RMSE): 4.228803605900616
Epoch 2 [===========================>   ]  Error (RMSE): 3.045208283205785
Epoch 3 [===========================>   ]  Error (RMSE): 2.3158002579709103
Epoch 4 [===========================>   ]  Error (RMSE): 1.8011619035535307
Epoch 5 [===========================>   ]  Error (RMSE): 1.4130705050184147
Epoch 6 [===========================>   ]  Error (RMSE): 1.1081507787050864
Epoch 7 [===========================>   ]  Error (RMSE): 0.8611078226567805
Epoch 8 [===========================>   ]  Error (RMSE): 0.6550586106606758
Epoch 9 [===========================>   ]  Error (RMSE): 0.4767350541886504


In [ ]:
for i in [2,5,11,99, 235]:
  print(m2.predict_index(i, 10))

[2157, 3047, 4697, 2714, 965, 4115, 4988, 1801, 3152, 3457]
[965, 4522, 3554, 3047, 2666, 3746, 2714, 2986, 1241, 1319]
[2615, 359, 1255, 739, 3722, 1287, 1795, 2851, 1305, 2610]
[965, 2714, 3001, 3152, 300, 3880, 4522, 4115, 3554, 4755]
[1114, 3152, 2666, 4755, 3047, 2157, 4416, 3533, 3457, 4910]


In [ ]:
# users that rated the first item of k = 500 vs k = 400
print('Users rated first item of prediction of k = 400 for user 2')
for i in arr[:,3184]:
  if i != 0:
    print(i, end = ' ')

print('\nUsers rated first item of prediction of k = 500 for user 2')
for i in arr[:,2157]:
  if i != 0:
    print(i, end = ' ')

Users rated first item of prediction of k = 400 for user 2
5 2 
Users rated first item of prediction of k = 500 for user 2
5 4 

### From above we can see, more users were considered when value of k is high.

In [ ]:
# Checking the diversity in predictions for k = 400 and k = 500
s1, c = 0, 0
for i in combinations([3184, 2766, 3871, 3505, 2732, 3987, 1155, 2862, 430, 3478], 2):
  s1 += np.linalg.norm(arr[:,i[0]] - arr[:,i[1]])
  c += 1

print(f'For k = 400 diversity is {s1/c}')

s1, c = 0, 0
for i in combinations([2157, 3047, 4697, 2714, 965, 4115, 4988, 1801, 3152, 3457], 2):
  s1 += np.linalg.norm(arr[:,i[0]] - arr[:,i[1]])
  c += 1

print(f'For k = 500 diversity is {s1/c}')

For k = 400 diversity is 7.684945050191584
For k = 500 diversity is 4.205925695214324


### From the above we can say that as we increase the k, diversity is reduced.